In [ ]:
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
from pylab import rcParams
from sklearn.metrics import confusion_matrix
import pygraphviz as pgv
from pyvis.network import Network
import matplotlib.colors as pltcol
from matplotlib.pyplot import figure
from IPython.display import display

In [ ]:
years = range(1990,2022)
for y in years:
    locals()['Gorg' + str(y)] = nx.read_gpickle('orgorg_single' +
                                               str(y) +
                                               '.gpickle')

In [ ]:
for i in tqdm(range(1990,2021)):
    nx.set_node_attributes(locals()['Gorg' + str(i)], pd.Series(nodes.node, index=nodes.id).to_dict(), 'name')
    nx.set_node_attributes(locals()['Gorg' + str(i)], pd.Series(nodes.country, index=nodes.id).to_dict(), 'country')
    nx.set_node_attributes(locals()['Gorg' + str(i)], pd.Series(nodes.continent, index=nodes.continent).to_dict(), 'continent')
    nx.set_node_attributes(locals()['Gorg' + str(i)], pd.Series(nodes.industry, index=nodes.id).to_dict(), 'industry')
    nx.set_node_attributes(locals()['Gorg' + str(i)], pd.Series(nodes.foundation, index=nodes.id).to_dict(), 'foundation')
    nx.set_node_attributes(locals()['Gorg' + str(i)], pd.Series(nodes.bipartite, index=nodes.id).to_dict(), 'bipartite')
    print(i)

In [ ]:
feat = 'country'
feat2 = 'industry'
edges = nx.to_pandas_edgelist(Gorg2020)
df = pd.merge(edges, nodes[[feat, feat2, 'id']], how = 'left', left_on = 'source', right_on = 'id' )
df2 =pd.merge(df, nodes[[feat, feat2, 'id']], how = 'left', left_on = 'target', right_on = 'id' )
weight = []
sub = df2[[feat+'_x', feat+'_y', feat2+'_x', feat2+'_y']]
sub2 = df2[[feat+'_x', feat+'_y', feat+'_x', feat+'_y']].drop_duplicates()
sub3 = sub2.dropna()
for i in tqdm(range(0,len(sub3))):
    
    sub4 = sub[((sub[feat+'_x']== sub3[feat+'_x'].values[i]) & (sub[feat+'_y']== sub3[feat+'_y'].values[i]))| ((sub[feat+'_y']== sub3[feat+'_x'].values[i]) & (sub[feat+'_x']== sub3[feat+'_y'].values[i]))  ]
    peso = len(sub4)
    weight.append(peso)
sub3['weight'] = weight
df3 = pd.merge(df2, sub3, left_on = [feat+'_x',feat+'_y'], right_on = [feat+'_x', feat+'_y'])
df4 = df3[df3[feat+'_x']<=df3[feat+'_y']]
df4['weight_l'] = np.log(df4['weight']+3)


In [ ]:
G = nx.from_pandas_edgelist(df4, feat+'_x', feat+'_y', edge_attr = 'weight_l')
nodebig = [20*np.sqrt(len(df4[(df4[(feat +'_x')]== i)| (df4[(feat +'_x')]== i)])) for i in list(G.nodes())]
d = dict(G.degree)
nodedeg = [d[k]*30 for k in d]
pos = nx.spring_layout(G, scale=100, k=20/np.sqrt(G.order()))

lung = len(G.edges)
edgewidth = [list(list(G.edges(data= True))[i][2].values())[0] for i in range(0,lung-1)]
fig, ax = plt.subplots(figsize=(16, 16))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
# Visualize graph components
nx.draw_networkx_edges(G, pos, alpha=0.2, width=edgewidth, edge_color="tab:blue")
nx.draw_networkx_nodes(G, pos, node_size=nodebig, node_color="tab:blue", alpha=1)
label_options = {"ec": "k", "fc": "white", "alpha": 0.7}
nx.draw_networkx_labels(G, pos, font_size=14, bbox=label_options)